In [2]:
!pip install deepface
!pip numpy
!pip pandas
!pip opencv-python
!pip matplotlib
!pip tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=4c7da043435221c6435a3c63a47bf5f0e9de9ac3cd0d6c4e292c6fafe2d9c4b4
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
ERROR: unknown command "numpy"
ERROR: unknown command "pandas"
ERROR: unknown command "opencv-python"
ERROR: unknown command "matplotlib"
ERROR: unknown command "tf-keras"


In [13]:
from deepface import DeepFace
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil

In [14]:
# List of available backends, models, and distance metrics
backends = ["opencv", "ssd", "dlib", "mtcnn", "retinaface"]
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
metrics = ["cosine", "euclidean", "euclidean_l2"]

In [15]:
import zipfile
with zipfile.ZipFile('Data.zip', 'r') as zip_ref:
  zip_ref.extractall('Data')

In [19]:
# Path to the image for face recognition
img = "Data/Data/klaus.png"
output_folder = "matched_images"
db_path = 'Data/Data/'

os.makedirs(output_folder, exist_ok=True)

def face_recognition(img):
    # Perform face recognition on the provided image
    # Find faces and identify people using a specific model and distance metric
    people = DeepFace.find(img_path=img, db_path=db_path, model_name=models[2], distance_metric=metrics[1])

    # Display the original image
    # plt.imshow(cv2.imread(img))

    # Print the identities of the recognized people
    for person in people:
      print(f'------------------ {person}')
      print(person['identity'][0].split('/')[1])


    for i, person_df in enumerate(people):
        print(f'-------- Matches for face {i + 1}: {len(person_df)} match(es) found --------')
        for index, row in person_df.iterrows():
            identity_path = row['identity']
            print(identity_path)


def face_recognition_v2(img):
    # Perform face recognition using DeepFace
    results = DeepFace.find(img_path=img, db_path=db_path, model_name=models[2], distance_metric=metrics[1])

    # Get base name of query image (e.g., "klaus.png")
    query_image_name = os.path.basename(img)

    # results is a list of DataFrames (one for each face found)
    for df in results:
        for _, row in df.iterrows():
            matched_path = row['identity']
            matched_filename = os.path.basename(matched_path)

            # Skip the query image itself
            if matched_filename == query_image_name:
                continue

            # Copy matched file to output folder
            destination_path = os.path.join(output_folder, matched_filename)
            shutil.copy(matched_path, destination_path)
            print(f"Matched and copied: {matched_filename}")

# Perform face recognition on a single image
face_recognition_v2(img)

25-05-30 12:10:46 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 1/1 [00:04<00:00,  4.84s/it]


25-05-30 12:10:51 - There are now 14 representations in ds_model_facenet512_detector_opencv_aligned_normalization_base_expand_0.pkl
25-05-30 12:10:51 - Searching Data/Data/klaus.png in 14 length datastore
25-05-30 12:10:52 - find function duration 6.3281872272491455 seconds
Matched and copied: klaus_family.png


In [ ]:
def realtime_face_recognition():
    # Define a video capture object
    vid = cv2.VideoCapture(0)

    while True:
        # Capture the video frame by frame
        ret, frame = vid.read()

        # Perform face recognition on the captured frame
        # Find faces and identify people using a specific model and distance metric
        people = DeepFace.find(img_path=frame, db_path=db_path, model_name=models[2], distance_metric=metrics[2], enforce_detection=False)

        for person in people:
            # Retrieve the coordinates of the face bounding box
            x = person['source_x'][0]
            y = person['source_y'][0]
            w = person['source_w'][0]
            h = person['source_h'][0]

            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Get the person's name and display it on the image
            name = person['identity'][0].split('/')[1]
            cv2.putText(frame, name, (x, y), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)

        # Display the resulting frame
        cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('frame', 960, 720)
        cv2.imshow('frame', frame)

        # Check if the 'q' button is pressed to quit the program
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close all windows
    vid.release()
    cv2.destroyAllWindows()

# Perform real-time face recognition using the webcam
# realtime_face_recognition()